Porcentaje de personas que se dedican a actividades de cuidados y no reciben remuneración

In [1]:
##
rm(list=ls())
#Paquetes y liberías
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, readr, stringr, dplyr, utils, survey)
# URL del archivo ZIP
url <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"

Cargando paquete requerido: pacman



In [2]:
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)

In [7]:
pob <- read_csv(fpath, show_col_types = FALSE)
names(pob)


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


[1] "folioviv"   "foliohog"   "numren"     "parentesco" "sexo"      
  [6] "edad"       "madre_hog"  "madre_id"   "padre_hog"  "padre_id"  
 [11] "disc_camin" "disc_ver"   "disc_brazo" "disc_apren" "disc_oir"  
 [16] "disc_vest"  "disc_habla" "disc_acti"  "cau_camin"  "cau_ver"   
 [21] "cau_brazo"  "cau_apren"  "cau_oir"    "cau_vest"   "cau_habla" 
 [26] "cau_acti"   "hablaind"   "lenguaind"  "hablaesp"   "comprenind"
 [31] "etnia"      "alfabetism" "asis_esc"   "nivel"      "grado"     
 [36] "tipoesc"    "tiene_b"    "otorg_b"    "forma_b"    "tiene_c"   
 [41] "otorg_c"    "forma_c"    "nivelaprob" "gradoaprob" "antec_esc" 
 [46] "residencia" "edo_conyug" "pareja_hog" "conyuge_id" "segsoc"    
 [51] "ss_aa"      "ss_mm"      "redsoc_1"   "redsoc_2"   "redsoc_3"  
 [56] "redsoc_4"   "redsoc_5"   "redsoc_6"   "hor_1"      "min_1"     
 [61] "usotiempo1" "hor_2"      "min_2"      "usotiempo2" "hor_3"     
 [66] "min_3"      "usotiempo3" "hor_4"      "min_4"      "usotiempo4"
 [71] "hor_5"      "min_5"      "usotiempo5" "hor_6"      "min_6"     
 [76] "usotiempo6" "hor_7"      "min_7"      "usotiempo7" "hor_8"     
 [81] "min_8"      "usotiempo8" "pop_insabi" "atemed"     "inst_1"    
 [86] "inst_2"     "inst_3"     "inst_4"     "inst_5"     "inst_6"    
 [91] "inscr_1"    "inscr_2"    "inscr_3"    "inscr_4"    "inscr_5"   
 [96] "inscr_6"    "inscr_7"    "inscr_8"    "prob_anio"  "prob_mes"  
[101] "prob_sal"   "aten_sal"   "servmed_1"  "servmed_2"  "servmed_3" 
[106] "servmed_4"  "servmed_5"  "servmed_6"  "servmed_7"  "servmed_8" 
[111] "servmed_9"  "servmed_10" "servmed_11" "servmed_12" "hh_lug"    
[116] "mm_lug"     "hh_esp"     "mm_esp"     "pagoaten_1" "pagoaten_2"
[121] "pagoaten_3" "pagoaten_4" "pagoaten_5" "pagoaten_6" "pagoaten_7"
[126] "noatenc_1"  "noatenc_2"  "noatenc_3"  "noatenc_4"  "noatenc_5" 
[131] "noatenc_6"  "noatenc_7"  "noatenc_8"  "noatenc_9"  "noatenc_10"
[136] "noatenc_11" "noatenc_12" "noatenc_13" "noatenc_14" "noatenc_15"
[141] "noatenc_16" "norecib_1"  "norecib_2"  "norecib_3"  "norecib_4" 
[146] "norecib_5"  "norecib_6"  "norecib_7"  "norecib_8"  "norecib_9" 
[151] "norecib_10" "norecib_11" "norecib_12" "razon_1"    "razon_2"   
[156] "razon_3"    "razon_4"    "razon_5"    "razon_6"    "razon_7"   
[161] "razon_8"    "razon_9"    "razon_10"   "razon_11"   "diabetes"  
[166] "pres_alta"  "peso"       "segvol_1"   "segvol_2"   "segvol_3"  
[171] "segvol_4"   "segvol_5"   "segvol_6"   "segvol_7"   "hijos_viv" 
[176] "hijos_mue"  "hijos_sob"  "trabajo_mp" "motivo_aus" "act_pnea1" 
[181] "act_pnea2"  "num_trabaj" "c_futuro"   "ct_futuro"  "entidad"   
[186] "est_dis"    "upm"        "factor"

In [12]:
datos <- pob[c("folioviv", "foliohog", "hor_4", "min_4", "est_dis", "entidad", "edad", "upm", "factor")]

datos <- datos %>%
  filter(entidad == "09") %>%
  mutate(
    pob_interes = case_when(
      (hor_4 != 0 | !is.na(hor_4)) ~ 1,
      (hor_4 == 0 & min_4 != 0 & !is.na(min_4)) ~ 1,
      TRUE ~ 0
    ) 
  ) 


In [13]:
## Diseño muestral
dm <- datos %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE)

In [14]:
pob_cuidados<- dm|>
  group_by(pob_interes)|>
  summarise(pob_cuid = survey_total(vartype = "cv"),
            porc_cuid = survey_prop(vartype = "cv")*100)|>
  #mutate(prc_p_cuid =pob_cuid*100/sum(pob_cuid))|> PUEDES OBTENER EL PORCENTAJE DE FORMA TRADICIONAL
  filter(pob_interes == 1)

In [15]:
head(pob_cuidados)


pob_interes,pob_cuid,pob_cuid_cv,porc_cuid,porc_cuid_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1569286,0.05277464,16.78938,4.772984
